#### Exercícios Busca Informada


#### Questão 1: Busca Gulosa

- Implementar a função heurística que considera o número de blocos na posição errada para o tabuleiro a seguir:

<img src = "images/exercicio.png" width = 200>

#### Questão 2: Busca A*

- Implementar o mesmo problema de Busca A* para o problema das Rotas na Romênia 

<img src = "images/BuscaEstrela.jpg" width = 1000  height = 500>


Dica: Não é necessário modificar as classes e estruturas. Basta chama-las para resolução do problema



#### Classes do Algoritmo de Busca Gulosa

In [ ]:
import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        '''
        Construtor
        Args:
            - num_blocos: numero de blocos por linha e coluna, valor inteiro (Ex: 3 significa 3 linhas e 3 colunas)
        '''
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        '''
        Verifica se dois estados são iguais
        Args:
            - atual: matriz que descreve o estado atual
            - objetivo: matriz que descreve o estado objetivo
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        flag = True
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False
                    break

        return flag

    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados
    
    
    def distancia_Manhattan(self, atual, objetivo):
        '''
        Calcula a distância de Manhattan entre o estado atual e o estado objetivo
        Args: 
            - atual: estado atual do tabuleiro
            - objetivo: estado objetivo do problema
        Return:
            - Distância de Manhattan
        '''
        
        distancia_total = 0
        
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                
                bloco_atual = atual[i, j]
                linha, coluna = self._encontra_posicao(objetivo, bloco_atual)
                
                distancia = abs(linha - i) + abs(coluna - j)
                distancia_total += distancia
                
        return distancia_total


import heapq

class BuscaGulosa():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def _verifica_visitado(self, estado, estados_visitados):
        '''
        Verifica se 'estado' está na lista de estados visitados
        Args:
            - estado: estado qualquer do tabuleiro
            - estados_visitados: lista com todos os estados já visitados
        Return:
            - booleano dizendo se o estado foi visitado ou não
        '''
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        
        '''
        OBS.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
        a prioridade desse estado
        '''
        
        p_fila = []
        
        # H, ID, elemento
        id_estado = 0
        heapq.heappush(p_fila, (0, id_estado, inicio))

        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)
            print(f"Estado:\n{atual}")

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estados(atual)
                print("Estados Gerados:")
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        id_estado += 1
                        distancia = self.problema.distancia_Manhattan(i, fim)
                        print(i)
                        print(distancia)
                        heapq.heappush(p_fila, (distancia, id_estado, i))

        return solucao_encontrada, estados_visitados, cont_estados


#### Classes para o Algoritmo de Busca A*

In [ ]:
class Graph:


    #Construtor recebendo um dicionário e colocando o grafo como direcionado
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    #faz com que o grafo se torne não-direcionado
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

    #Conecta dois nós no grafo
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance

    #Recupera as conexões de um nó A
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    #Retorna uma lista de todos os Nós no Grafo
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

#Classe responsável pelos nós
class Node:
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distancia para o nó de inicio
        self.h = 0 # Distancia para o nó objetivo
        self.f = 0 # Custo Total (soma de G e H. Basicamente, trata-se da função f(n) referida)

    '''Este método é chamado quando você compara dois objetos da classe usando o operador de igualdade ==.
    Ele retorna True se os nomes dos dois nós forem iguais e False caso contrário.'''
    def __eq__(self, other):
        return self.name == other.name
    
    '''Este método é chamado quando você compara dois objetos da classe usando o operador <.
    Ele retorna True se o custo total (f) do nó atual for menor do que o custo total do outro nó.'''
    def __lt__(self, other):
         return self.f < other.f
    
    '''Este método é chamado quando você chama a função repr() em um objeto da classe ou quando usa print() para imprimir o objeto.
    Ele retorna uma representação em string do objeto, neste caso, uma string no formato (name,f).'''
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))

    '''Este método é chamado quando você chama a função hash() em um objeto da classe.
    Ele retorna um valor de hash para o objeto, neste caso, o hash do nome do nó.'''
    def __hash__(self):
        return hash(self.name)
    

In [ ]:
import heapq
import numpy as np

def get_path(atual, inicial):
    '''
    Função para gerar o caminho percorrido entre dois estados
    Args:
        - atual: estado atual
        - inicial: estado inicial
    Return:
        - Caminho percorrido entre o estado inicial e final
    '''
    path = []
    while atual != inicial:
        path.append(atual.name + ': ' + str(atual.g))
        atual = atual.parent
    path.append(inicial.name + ': ' + str(inicial.g))

    return path[::-1]
        

def a_star(grafo, heuristicas, inicio, fim):
    '''
    Algoritmo a*, dado um grafo, um cojunto de heurísticas, um estado/nó inicial e um estado/nó final, 
    busca o caminho ótimo e completo
    Args:
        - grafo: grafo contendo os estados da busca
        - heuristicas: heuristica de cada estado
        - inicio: estado inicial
        - fim: estado final
    Return:
        - Melhor caminho entre estados inicial e final, se a solução foi encontrada, quantidade de estados visitados
    '''
    
    # Criando nó inicial e nó objetivo (não possuem pai)
    start_node = Node(inicio, None)
    goal_node = Node(fim, None)
    
    # Inicializando atributos g(0) = 0,  f(0) = 0 + h(0)
    start_node.g = 0
    start_node.h = heuristicas[inicio]
    start_node.f = start_node.g + start_node.h

    state_id = 0

    # Fila de prioridades
    fila = []
    heapq.heappush(fila, (start_node.f, state_id, start_node))

    # Nós criados
    created_nodes = []
    created_nodes.append(start_node)

    # Nós visitados
    visited_nodes = []
    
    # Controle
    solucao_encontrada = False
    path = None

    while len(fila) != 0:
        print("="*50)
        atual = heapq.heappop(fila)[2]
        visited_nodes.append(atual)
        print(f"Visitando {atual.name} - H: {atual.h} - G: {atual.g} - F: {atual.f}")

        if atual == goal_node:
            path = get_path(atual, start_node)
            solucao_encontrada = True
            break

        # Percorrendo todos os vizinhos do nó atual
        for nome_viz, dist_viz in grafo.get(atual.name).items():

            # Criando nó com o nome do vizinho e passando o estado atual como pai
            vizinho = Node(nome_viz, atual)

            # Verificar se a distância pai -> vizinho (g) é menor que 
            # um possível valor (g) do vizinho (caso já tenha sido visitado)

            # caso não haja valor de g para o estado atual a função retorna infinito
            g_pai_vizinho = atual.g + dist_viz
            
            g_vizinho = vizinho.g if vizinho in created_nodes else np.inf
                
            # Conectando pai ao novo nó
            if g_pai_vizinho < g_vizinho:
                
                vizinho.g = g_pai_vizinho
                vizinho.h = heuristicas[vizinho.name]
                vizinho.f = vizinho.g + vizinho.h
                
                created_nodes.append(vizinho)
                state_id += 1
                heapq.heappush(fila, (vizinho.f, state_id, vizinho))
    
        print("Estados disponíveis:")
        print(fila)
        
    return path, solucao_encontrada, visited_nodes 

#### Questão 1

#### Questão 2